In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from pathlib import Path

pd.options.display.float_format = '{:,.4f}'.format
root_dir = str(Path.cwd().parent.parent)

fontprop = {'family': 'serif'}

# Precisão

In [3]:
def get_results_precision(filename):
    data = pd.read_csv(root_dir + "/data/experimental_results/precision/" + filename, index_col=0, float_precision='high')
    data = data[['mad_len', 'std_dev_len', 'mad_az', 'std_dev_az', 'mad_el', 'std_dev_el']]
    print(data)

In [4]:
get_results_precision("fixed_1.csv")
get_results_precision("fixed_2.csv")
get_results_precision("fixed_3.csv")

           mad_len  std_dev_len  mad_az  std_dev_az  mad_el  std_dev_el
technique                                                              
music       4.1975       8.6128  2.0365      8.5105  4.7399      6.5641
esprit      4.6589       6.4586  1.8465      6.0749  5.3191      7.1224
silabs      0.9300       1.1788  0.9057      1.1539  1.7972      2.2666
           mad_len  std_dev_len  mad_az  std_dev_az  mad_el  std_dev_el
technique                                                              
music       3.4977      10.9712  2.8472     11.0574  5.8038      7.2491
esprit      3.7891       5.9085  2.3586      6.6148  4.3858      6.0278
silabs      1.3358       1.6246  1.3659      1.6542  1.9405      2.4688
           mad_len  std_dev_len  mad_az  std_dev_az  mad_el  std_dev_el
technique                                                              
music       2.0604       6.0833  1.8215      6.5930  1.7973      2.9670
esprit      2.2989       6.3690  2.1553     11.6199  3.0095     

# Hiperparâmetros

## Busca grossa-fina

In [5]:
data = pd.read_csv(root_dir + "/data/experimental_results/hyperparameters/" + "coarse_fine_search.csv", index_col=0, float_precision='high')
data = data[['runtime', 'mae_len','rmse_len', 'rmse_len_99p']]
data['runtime'] = data['runtime'] / 1000

In [6]:
runtime_data = data['runtime'].values
mae_data = data['mae_len'].values
index = data.index.values
fig = plt.figure(figsize=(7,4), dpi=300)
ax1 = plt.axes()
ax2 = ax1.twinx()
lgd1 = ax1.plot(index, runtime_data, 'b-', label='Tempo ' + r'$ \alpha $' +  ' estático')
lgd2 = ax2.plot(index, mae_data, 'b--', label='MAE ' + r'$ \alpha $' +  ' estático')
ax1.set_xlabel('Passo Grosso (°)', fontprop)
ax1.set_ylabel('Tempo (s)', fontprop)
ax2.set_ylabel('MAE (°)', fontprop)
lgd = lgd1 + lgd2
labels = [l.get_label() for l in lgd]
ax1.legend(lgd, labels, loc='upper left', prop=fontprop)
plt.savefig("busca_grossa_fina_pg.png", bbox_inches='tight', pad_inches=0)
plt.close()

In [7]:
print(data)

             runtime  mae_len  rmse_len  rmse_len_99p
coarse_step                                          
2            42.9763   0.1687    1.6013        0.1330
3            21.6049   0.1944    1.8114        0.1350
4            15.5510   0.2845    4.1453        0.1384
5            13.7561   0.4787    9.0651        0.1381
6            14.3563   0.4797    7.3349        0.1365
7            16.2182   0.7028    9.2024        0.1990
8            19.0011   0.7801    8.8870        0.2136
9            22.1796   1.3267   14.5058        0.9829


## Gradiente Simples com learning rate fixo e adaptável

In [8]:
data_f = pd.read_csv(root_dir + "/data/experimental_results/hyperparameters/" + "gradient_simple.csv", index_col=0, float_precision='high')
data_f = data_f[['learning_rate', 'runtime', 'mae_len', 'rmse_len', 'rmse_len_99p']]
data_f['runtime'] = data_f['runtime'] / 1000
data_a = pd.read_csv(root_dir + "/data/experimental_results/hyperparameters/" + "gradient_adapt_lr.csv", index_col=0, float_precision='high')
data_a = data_a[['learning_rate', 'runtime', 'mae_len', 'rmse_len', 'rmse_len_99p']]
data_a['runtime'] = data_a['runtime'] / 1000

### Alfa

In [9]:
# Fix
data_sliced = data_f.loc[(data_f.index == 5)]
runtime_data = data_sliced['runtime'].values
mae_data = data_sliced['mae_len'].values
index = data_sliced['learning_rate'].values
fig = plt.figure(figsize=(7,4), dpi=300)
ax1 = plt.axes()
ax2 = ax1.twinx()
ax1.plot(index, runtime_data, 'b-', label='Tempo ' + r'$ \alpha $' +  ' estático')
ax2.plot(index, mae_data, 'b--', label='MAE ' + r'$ \alpha $' +  ' estático')
ax1.set_xlabel('$\it{Learning\ rate}$', fontprop)
ax1.set_ylabel('Tempo (s)', fontprop)
ax2.set_ylabel('MAE (°)', fontprop)
# Adapt
data_sliced = data_a.loc[(data_a.index == 5)]
runtime_data = data_sliced['runtime'].values
mae_data = data_sliced['mae_len'].values
index = data_sliced['learning_rate'].values
ax1.plot(index, runtime_data, 'r-', label='Tempo ' + r'$ \alpha $' +  ' Adaptável')
ax2.plot(index, mae_data, 'r--', label='MAE ' + r'$ \alpha $' +  ' Adaptável')
ax1.legend(loc='upper left', prop=fontprop)
ax2.legend(loc='upper right', prop=fontprop)
plt.savefig("gradiente_simples_alfa.png", bbox_inches='tight', pad_inches=0)
plt.close()

### Passo Grosso

In [10]:
# Fix
data_sliced = data_f.loc[(data_f['learning_rate'] == 0.1)]
runtime_data = data_sliced['runtime'].values
mae_data = data_sliced['mae_len'].values
index = data_sliced.index.values
fig = plt.figure(figsize=(7,4), dpi=300)
ax1 = plt.axes()
ax2 = ax1.twinx()
ax1.plot(index, runtime_data, 'b-', label='Tempo ' + r'$ \alpha $' +  ' estático')
ax2.plot(index, mae_data, 'b--', label='MAE ' + r'$ \alpha $' +  ' estático')
ax1.set_xlabel('Passo Grosso (°)', fontprop)
ax1.set_ylabel('Tempo (s)', fontprop)
ax2.set_ylabel('MAE (°)', fontprop)
# Adapt
data_sliced = data_a.loc[(data_a['learning_rate'] == 0.7)]
runtime_data = data_sliced['runtime'].values
mae_data = data_sliced['mae_len'].values
index = data_sliced.index.values
ax1.plot(index, runtime_data, 'r-', label='Tempo ' + r'$ \alpha $' +  ' Adaptável')
ax2.plot(index, mae_data, 'r--', label='MAE ' + r'$ \alpha $' +  ' Adaptável')
ax1.legend(loc='upper left', prop=fontprop)
ax2.legend(loc='upper right', prop=fontprop)
plt.savefig("gradiente_simples_pg.png", bbox_inches='tight', pad_inches=0)
plt.close()

### Melhores hiperparâmetros

#### LR Fixo

In [11]:
data_sliced = data_f.loc[(data_f['mae_len'] < 0.3)]
data_sliced = data_sliced.sort_values(by=['runtime', 'rmse_len'])
data_sliced = data_sliced[:5]
print(data_sliced)
data_sliced = data_f.loc[(data_f['rmse_len_99p'] < 0.3)]
data_sliced = data_sliced.sort_values(by=['runtime', 'rmse_len'])
data_sliced = data_sliced[:5]
print(data_sliced)

             learning_rate  runtime  mae_len  rmse_len  rmse_len_99p
coarse_step                                                         
5                   0.1000  11.1536   0.2071    4.1054        0.1155
5                   0.0900  11.3634   0.2031    4.1046        0.1105
5                   0.0700  12.1642   0.1995    4.1069        0.1037
5                   0.0500  13.8613   0.1994    4.1134        0.1002
3                   0.1000  22.1204   0.2937    8.2159        0.1109
             learning_rate  runtime  mae_len  rmse_len  rmse_len_99p
coarse_step                                                         
11                  0.1000   7.8568   1.0721   14.2961        0.2689
11                  0.0900   8.2288   1.0765   14.3063        0.2456
8                   0.1000   8.5228   0.5167    8.1139        0.1796
12                  0.0900   8.5654   1.4533   17.6927        0.2909
8                   0.0900   8.8337   0.5202    8.1318        0.1643


#### LR Adaptável

In [12]:
data_sliced = data_a.loc[(data_a['mae_len'] < 0.3)]
data_sliced = data_sliced.sort_values(by=['runtime', 'rmse_len'])
data_sliced = data_sliced[:5]
print(data_sliced)
data_sliced = data_a.loc[(data_a['rmse_len_99p'] < 0.3)]
data_sliced = data_sliced.sort_values(by=['runtime', 'rmse_len'])
data_sliced = data_sliced[:5]
print(data_sliced)

             learning_rate  runtime  mae_len  rmse_len  rmse_len_99p
coarse_step                                                         
5                   0.8000   8.0548   0.1903    4.0938        0.0982
5                   0.7000   8.1033   0.1907    4.0940        0.0989
5                   0.6000   8.1147   0.1913    4.0941        0.0991
5                   0.5000   8.1927   0.1923    4.0946        0.0994
5                   0.4000   8.3430   0.1936    4.0957        0.0997
             learning_rate  runtime  mae_len  rmse_len  rmse_len_99p
coarse_step                                                         
14                  0.8000   2.7910   1.6515   17.6669        0.2841
13                  0.9000   2.8062   1.2990   14.6818        0.1965
13                  0.8000   2.8364   1.2993   14.6818        0.1968
14                  0.7000   2.8814   1.6520   17.6670        0.2850
13                  0.7000   2.9322   1.2998   14.6819        0.1982


## Momentum e Nesterov

In [13]:
data_m = pd.read_csv(root_dir + "/data/experimental_results/hyperparameters/" + "gradient_momentum.csv", index_col=0, float_precision='high')
data_m = data_m[['learning_rate', 'momentum', 'runtime', 'mae_len', 'rmse_len', 'rmse_len_99p']]
data_m = data_m.loc[(data_m['learning_rate'] >= 0.01)]
data_m['runtime'] = data_m['runtime'] / 1000
data_n = pd.read_csv(root_dir + "/data/experimental_results/hyperparameters/" + "gradient_nesterov.csv", index_col=0, float_precision='high')
data_n = data_n[['learning_rate', 'momentum', 'runtime', 'mae_len', 'rmse_len', 'rmse_len_99p']]
data_n = data_n.loc[(data_n['learning_rate'] >= 0.01)]
data_n['runtime'] = data_n['runtime'] / 1000

### Beta

In [14]:
# Momentum
data_sliced = data_m.loc[(data_m.index == 5) & (data_m['learning_rate'] == 0.07)]
runtime_data = data_sliced['runtime'].values
mae_data = data_sliced['mae_len'].values
index = data_sliced['momentum'].values
fig = plt.figure(figsize=(7,4), dpi=300)
ax1 = plt.axes()
ax2 = ax1.twinx()
ax1.plot(index, runtime_data, 'b-', label='Tempo Momentum')
ax2.plot(index, mae_data, 'b--', label='MAE Momentum')
ax1.set_xlabel('Momentum', fontprop)
ax1.set_ylabel('Tempo (s)', fontprop)
ax2.set_ylabel('MAE (°)', fontprop)
# Nesterov
data_sliced = data_n.loc[(data_n.index == 5) & (data_n['learning_rate'] == 0.07)]
runtime_data = data_sliced['runtime'].values
mae_data = data_sliced['mae_len'].values
index = data_sliced['momentum'].values
ax1.plot(index, runtime_data, 'r-', label='Tempo Nesterov')
ax2.plot(index, mae_data, 'r--', label='MAE Nesterov')
ax1.legend(loc='upper left', prop=fontprop)
ax2.legend(loc='upper right', prop=fontprop)
plt.savefig("momentum_beta.png", bbox_inches='tight', pad_inches=0)
plt.close()

### Alfa

In [15]:
# Momentum
data_sliced = data_m.loc[(data_m.index == 5) & (data_m['momentum'] == 0.9)]
runtime_data = data_sliced['runtime'].values
mae_data = data_sliced['mae_len'].values
index = data_sliced['learning_rate'].values
fig = plt.figure(figsize=(7,4), dpi=300)
ax1 = plt.axes()
ax2 = ax1.twinx()
ax1.plot(index, runtime_data, 'b-', label='Tempo Momentum')
ax2.plot(index, mae_data, 'b--', label='MAE Momentum')
ax1.set_xlabel('$\it{Learning\ rate}$', fontprop)
ax1.set_ylabel('Tempo (s)', fontprop)
ax2.set_ylabel('MAE (°)', fontprop)
# Nesterov
data_sliced = data_n.loc[(data_n.index == 5) & (data_n['momentum'] == 0.9)]
runtime_data = data_sliced['runtime'].values
mae_data = data_sliced['mae_len'].values
index = data_sliced['learning_rate'].values
ax1.plot(index, runtime_data, 'r-', label='Tempo Nesterov')
ax2.plot(index, mae_data, 'r--', label='MAE Nesterov')
ax1.legend(loc='upper left', prop=fontprop)
ax2.legend(loc='upper right', prop=fontprop)
plt.savefig("momentum_alfa.png", bbox_inches='tight', pad_inches=0)
plt.close()

### Passo Grosso

In [16]:
# Momentum
data_sliced = data_m.loc[(data_m['learning_rate'] == 0.07) & (data_m['momentum'] == 0.9)]
runtime_data = data_sliced['runtime'].values
mae_data = data_sliced['mae_len'].values
index = data_sliced.index.values
fig = plt.figure(figsize=(7,4), dpi=300)
ax1 = plt.axes()
ax2 = ax1.twinx()
ax1.plot(index, runtime_data, 'b-', label='Tempo Momentum')
ax2.plot(index, mae_data, 'b--', label='MAE Momentum')
ax1.set_xlabel('Passo Grosso (°)', fontprop)
ax1.set_ylabel('Tempo (s)', fontprop)
ax2.set_ylabel('MAE (°)', fontprop)
# Nesterov
data_sliced = data_n.loc[(data_n['learning_rate'] == 0.07) & (data_n['momentum'] == 0.9)]
runtime_data = data_sliced['runtime'].values
mae_data = data_sliced['mae_len'].values
index = data_sliced.index.values
ax1.plot(index, runtime_data, 'r-', label='Tempo Nesterov')
ax2.plot(index, mae_data, 'r--', label='MAE Nesterov')
ax1.legend(loc='upper left', prop=fontprop)
ax2.legend(loc='upper right', prop=fontprop)
plt.savefig("momentum_pg.png", bbox_inches='tight', pad_inches=0)
plt.close()

### Melhores hiperparâmetros

#### Momentum

In [17]:
data_sliced = data_m.loc[(data_m['mae_len'] < 0.3)]
data_sliced = data_sliced.sort_values(by=['runtime', 'rmse_len'])
data_sliced = data_sliced[:5]
print(data_sliced)
data_sliced = data_m.loc[(data_m['rmse_len_99p'] < 0.3)]
data_sliced = data_sliced.sort_values(by=['runtime', 'rmse_len'])
data_sliced = data_sliced[:5]
print(data_sliced)

             learning_rate  momentum  runtime  mae_len  rmse_len  rmse_len_99p
coarse_step                                                                   
5                   0.1000    0.8000   8.1216   0.1909    4.0932        0.0986
5                   0.1000    0.8500   8.1477   0.1894    4.0940        0.0974
5                   0.0900    0.8000   8.1788   0.1912    4.0933        0.0989
5                   0.0900    0.8500   8.1819   0.1897    4.0936        0.0983
5                   0.1000    0.7500   8.1836   0.1912    4.0935        0.0989
             learning_rate  momentum  runtime  mae_len  rmse_len  rmse_len_99p
coarse_step                                                                   
13                  0.1000    0.8500   2.9089   1.2967   14.6823        0.1948
14                  0.0900    0.8500   2.9172   1.6500   17.6665        0.2839
14                  0.1000    0.8500   2.9179   1.6511   17.6664        0.2949
14                  0.1000    0.8000   2.9315   1.65

#### Nesterov

In [18]:
data_sliced = data_n.loc[(data_n['mae_len'] < 0.3)]
data_sliced = data_sliced.sort_values(by=['runtime', 'rmse_len'])
data_sliced = data_sliced[:5]
print(data_sliced)
data_sliced = data_n.loc[(data_n['rmse_len_99p'] < 0.3)]
data_sliced = data_sliced.sort_values(by=['runtime', 'rmse_len'])
data_sliced = data_sliced[:5]
print(data_sliced)

             learning_rate  momentum  runtime  mae_len  rmse_len  rmse_len_99p
coarse_step                                                                   
5                   0.0500    0.9000   8.3772   0.1899    4.0934        0.1010
5                   0.0500    0.8500   8.4950   0.1919    4.0933        0.1025
5                   0.0500    0.9500   8.5126   0.1840    4.0931        0.0929
5                   0.0700    0.9000   8.5766   0.1921    4.0948        0.1103
5                   0.0300    0.9000   8.5957   0.1903    4.0932        0.0989
             learning_rate  momentum  runtime  mae_len  rmse_len  rmse_len_99p
coarse_step                                                                   
14                  0.0500    0.9500   3.1806   1.6431   17.6657        0.2823
14                  0.0500    0.9000   3.1947   1.6502   17.6668        0.2863
14                  0.0700    0.9000   3.2433   1.6523   17.6662        0.2960
13                  0.0500    0.9500   3.2635   1.29

# Acurácia e Runtime

In [19]:
data = pd.read_csv(root_dir + "/data/experimental_results/accuracy_runtime.csv", index_col=0, float_precision='high')
data['runtime'] = data['runtime'] / 1000
data = data[['runtime', 'mae_len', 'rmse_len']]
print(data)

                                 runtime  mae_len  rmse_len
technique                                                  
esprit                            1.6537   0.0000    0.0000
music_coarse_fine_grid_search   108.1367   0.3367    7.2494
music_gradient_simple            56.8658   0.2845    6.2201
music_gradient_adapt_lr          39.9917   0.3205    7.2172
music_gradient_momentum          41.2383   0.2796    6.5136
music_gradient_nesterov          43.1562   0.2588    6.2032
music_fine_grid_search        8,917.4060   0.1963    5.0916
